# Quickstart
For the predictive calculation of the current ciritcality status of multiple traffic participants i.e. a traffic scene, future paths of the vehicles and their current state are of high importantness.  

For the estimation of the future paths the underlying road map is used. In the case no road information is supplied, the road geometry is treated as an unstructured space and the future paths are represented as straight lines.  
*(Using the calculation without any road information might affect the transferability of the results.)*

In [ ]:
from oncrit.utils.helpers import generate_metric_base

In the next step, the object lists in csv format and the map information in lanelet format are specified as the file path.  
Since the lanelet map is referenced in geodetic coordinates, a reference point (map_origin: lat, lon) is required to align it with the metric object list format.  
*Only lanelet is currently supported as map format.*

In [ ]:
object_list_file = "../tests/data/tracks_cam1_interpolated.csv"
osm_map = "../tests/data/k729_2022-03-16_fix.osm"
map_origin = (49.01160993928274, 8.43856470258739)


The scene information in form of an object list and the `case_id` + `frame_id`  is fed into the `generate_metric_base` helper which merges the traffic participants' states with the potential trajectory information. This step can also be done manually if the helper does not match the required dependency.

The `threshold` parameter (in m) is required for s-t metrics to determine how close a vehicle must be to the future path to be considered relevant. The higher the value, the more vehicles will be taken into account. If it is too high, vehicles that are not on the same track could be included and this could lead to false results. (Rule of thumb: `threshold ~ lane_width / 1.5`)

[Metrics](docs/metrics_explanation.md) can be calculated by calling the corresponding member function of the `metric_base`. The criticality information is returned in the coordinate-format (triplet format):

```
[
    (vehicle_ID_A, vehicle_ID_B, metric_value)`,
    (vehicle_ID_A, vehicle_ID_C, metric_value)`,
    (vehicle_ID_F, vehicle_ID_G, metric_value)`,
    ...
]
```

If you are confronted with results that do not match your expectations, check the consistency of your data and [common pitfals](tutorial_test_your_data.ipynb).

In [ ]:
metric_base = generate_metric_base(
    object_list_file=object_list_file, frame_id=1689, case_id=None, map_file=osm_map, map_origin=map_origin)

threshold = 2.5

print("EuclideanDistanceSimple", metric_base.euclidean_distance_simple())
print("EuclideanDistanceApproximated", metric_base.euclidean_distance_approximated())
print("TTC", metric_base.ttc(threshold))
print("Headway", metric_base.headway(threshold))
print("Gaptime", metric_base.gaptime(threshold))
print("Clearance", metric_base.clearance(threshold))
print("Spacing", metric_base.spacing(threshold))
print("PPET", metric_base.predictive_encroachment_time(threshold))